In [ ]:
import io, sys, os, datetime, requests, json
from collections import defaultdict
import numpy as np
import pandas as pd
import shapely
import geopandas as gpd
import geemap
import ee

In [ ]:
#ee.Authenticate()
ee.Initialize()

In [ ]:
with open('popcodes.json', 'r') as popfile:
    popcodes2 = json.loads(popfile.read())

In [ ]:
scale = 100

bycountry = {}
for year in [2010, 2015, 2020]:
    print('\n\n{0}\n'.format(year))
    urbext = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}'.format(year))
    urbext_data = geemap.ee_to_df(urbext)
    
    worldpop_ic = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.eq('year', year))
    
    excdays = ee.Image('projects/wri-datalab/SCL-Cities/cams-eac4-exceedancedays_any_v2_{0}'.format(year))
    
    exc_res = defaultdict(list)
    pop_res = defaultdict(list)
    excpop_total = []
    pop_total = []
    countries = []
    for i in range(len(urbext_data)):
        ua = urbext_data.iloc[i]
        ua_f = urbext.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes:
            countries.append(ua['country'])
            local_excdays = excdays.reduceRegion(ee.Reducer.mean(), geom, scale, maxPixels=1e12).get('b1').getInfo()
            ghspop_img = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP").filter(ee.Filter.calendarRange(year, year, 'year')).select('population_count').first()
            localpop = ghspop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('population_count').getInfo()
            
            exc_res[ua['country']].append(local_excdays)
            pop_res[ua['country']].append(localpop)
            excpop_total.append(local_excdays* localpop)
            pop_total.append(localpop)
            print(i, end=' ')
    
    if not 'global' in list(bycountry.keys()):
        bycountry['global'] = {}
    bycountry['global'] = bycountry['global'] | {
        'pop-averaged_exceedance_days_{0}'.format(year): sum(excpop_total) / sum(pop_total)
    }
    country_list = list(set(exc_res.keys()))
    country_list.sort()
    for country in country_list:
        for country in country_list:
            if len(country) > 2:
                excpop = 0
                for idx in range(len(exc_res[country])):
                    excpop += exc_res[country][idx] * pop_res[country][idx]
                if not country in list(bycountry.keys()):
                    bycountry[country] = {}
                bycountry[country] = bycountry[country] | {
                    'pop-averaged_exceedance_days_{0}'.format(year): excpop / sum(pop_res[country])
                }
res_pd = pd.DataFrame.from_dict(bycountry, orient='index')
res_pd.to_csv('CTY-10_air-pollutant-exceedance-days_v3.csv')